# Denver Crime Heatmap Notebook
CS 3120 Machine Learning Term Project - Konstantin Zaremski

---

This comprehensive notebook covers my project from ideation and planning, to exploratory data analysis (EDA), and final implementation.
### *********** Unfinished Draft (for peer review)

## Exploratory Data Analysis

### Setup
Importing required modules and libraries for EDA, etc. See `requirements_eda.txt` for a full list of requirements to install into the environment using `pip`.

In [11]:
from IPython.display import display
import pandas as pd
import pyproj
import numpy as np
import glob

### Load Crime Data

In [12]:
# Find all crime data CSV files matching the naming pattern
csv_files = glob.glob("data/crime_split_*.csv")

# Read in all of the files and create Pandas dataframes from them
split_dfs = [pd.read_csv(f) for f in csv_files]

# Concatenate all the dataframes into a single one
df = pd.concat(split_dfs, ignore_index=True)

display(df.head())

original_dataframe_shape = df.shape
print(f'Loaded {original_dataframe_shape[0]} records!')

,OBJECTID,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,VICTIM_COUNT,x,y
0,20000,2020467360,2020467360299901,2999,1,criminal-mischief-mtr-veh,public-disorder,8/2/2020 10:43:00 PM,NaN,8/2/2020 10:43:00 PM,...,-105.024597,39.689751,4,422,ruby-hill,1,0,1,3.133789e+06,1.676471e+06
1,20001,20196003434,20196003434299901,2999,1,criminal-mischief-mtr-veh,public-disorder,4/20/2019 7:30:00 AM,4/20/2019 8:45:00 AM,4/20/2019 6:30:00 PM,...,-104.987348,39.714316,3,311,speer,1,0,1,3.144221e+06,1.685476e+06
2,20002,2020123049,2020123049299901,2999,1,criminal-mischief-mtr-veh,public-disorder,2/25/2020 11:30:00 PM,2/26/2020 6:45:00 AM,2/26/2020 7:30:00 AM,...,-104.988098,39.764528,6,612,five-points,1,0,1,3.143907e+06,1.703765e+06
3,20003,2021621685,2021621685299901,2999,1,criminal-mischief-mtr-veh,public-disorder,11/1/2021 6:20:00 AM,11/1/2021 6:30:00 AM,11/1/2021 10:15:00 AM,...,-105.004665,39.739669,1,123,lincoln-park,1,0,1,3.139299e+06,1.694684e+06
4,20004,2020289138,2020289138299901,2999,1,criminal-mischief-mtr-veh,public-disorder,5/9/2020 12:00:00 PM,NaN,5/11/2020 12:05:00 PM,...,-104.830661,39.795281,5,521,montbello,1,0,1,3.188083e+06,1.715255e+06


Loaded 394736 records!


### Column Analysis
The next thing to do is determine which columns will be useful for predictions and which columns need to be discarded.

In [13]:
print("Crime Dataset Columns:")
for column in df.columns:
    print(f" - {column}")

Crime Dataset Columns:
 - OBJECTID
 - INCIDENT_ID
 - OFFENSE_ID
 - OFFENSE_CODE
 - OFFENSE_CODE_EXTENSION
 - OFFENSE_TYPE_ID
 - OFFENSE_CATEGORY_ID
 - FIRST_OCCURRENCE_DATE
 - LAST_OCCURRENCE_DATE
 - REPORTED_DATE
 - INCIDENT_ADDRESS
 - GEO_X
 - GEO_Y
 - GEO_LON
 - GEO_LAT
 - DISTRICT_ID
 - PRECINCT_ID
 - NEIGHBORHOOD_ID
 - IS_CRIME
 - IS_TRAFFIC
 - VICTIM_COUNT
 - x
 - y


#### Column Decisions
| Column ID                 | Decison   | Explanation                                                                                        |
|---------------------------|-----------|----------------------------------------------------------------------------------------------------|
| `OBJECTID`                | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `INCIDENT_ID`             | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_ID`              | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_CODE`            | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_CODE_EXTENSION`  | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_TYPE_ID`         | Kept      | This is part of the output (y-values). We will keep this column and it will be the subject of prediction by the classification model. |
| `OFFENSE_CATEGORY_ID`     | Discarded | This is part of the output (y-values). This will be discarded in  |
| `FIRST_OCCURRENCE_DATE`   | Kept      | Timestamp of the event, this will be kept and used for predictions made based on the time of day. |
| `LAST_OCCURRENCE_DATE`    | Discarded | Undefined for many rows. We will discard this timestamp and rely on `FIRST_OCCURRENCE_DATE` instead.                              |
| `REPORTED_DATE`           | Discarded | This is part of the output (y-values). Report time delta from occurrence varies incident to incident. No predictive value.                             |
| `INCIDENT_ADDRESS`        | Discarded | Although address is useful, this column is not standardized with entries like "2400 block" rather than exact addresses in some cases. |
| `GEO_X`                   | Discarded | Related to the mapping platform the data was posted on. No predictive value.                    |
| `GEO_Y`                   | Discarded | Related to the mapping platform the data was posted on. No predictive value.                    |
| `GEO_LON`                 | Kept      | Location of the crime, this will be kept and used to make predictions based on location. |
| `GEO_LAT`                 | Kept      | Location of the crime, this will be kept and used to make predictions based on location. |
| `DISTRICT_ID`             | Discarded | This is part of the output (y-values) and we have no way of querying. No predictive value.                    |
| `PRECINCT_ID`             | Discarded | This is part of the output (y-values) and we have no way of querying. No predictive value.                             |
| `NEIGHBORHOOD_ID`         | Discarded | Although neighborhood is useful, as some are "rougher" than others, we don't have a standard way of querying.                             |
| `IS_CRIME`                | Discarded | This is part of the output (y-values). No predictive value.                             |
| `IS_TRAFFIC`              | Discarded | This is part of the output (y-values). No predictive value.                             |
| `VICTIM_COUNT`            | Discarded | This is part of the output (y-values). No predictive value.                             |
| `x`                       | Discarded | Related to the mapping platform the data was posted on. No predictive value.                     |
| `y`                       | Discarded | Related to the mapping platform the data was posted on. No predictive value.                    |

In [14]:
# Keep only the columns that we want
df = df[['OFFENSE_TYPE_ID','FIRST_OCCURRENCE_DATE','GEO_LON','GEO_LAT']]

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=20))

,OFFENSE_TYPE_ID,FIRST_OCCURRENCE_DATE,GEO_LON,GEO_LAT
245977,theft-of-motor-vehicle,1/28/2024 12:16:00 PM,-104.895443,39.786789
87825,theft-parts-from-vehicle,12/16/2019 8:00:00 AM,-105.017614,39.775675
259603,theft-of-motor-vehicle,6/23/2019 8:00:00 PM,-104.910718,39.714221
9764,drug-heroin-possess,3/26/2020 11:00:00 PM,-105.057571,39.679808
335144,theft-of-motor-vehicle,1/21/2021 4:19:00 AM,-105.049212,39.632147
313307,animal-cruelty-to,12/8/2022 1:10:00 PM,-104.772255,39.798556
294871,criminal-mischief-other,8/28/2023 5:00:00 AM,-105.027641,39.657185
120600,criminal-mischief-other,8/25/2024 3:50:00 AM,-104.974766,39.728934
302441,violation-of-court-order,9/24/2023 7:00:00 PM,-105.001113,39.782194
170846,theft-parts-from-vehicle,9/3/2021 3:25:00 PM,-105.006580,39.715154


### Time Binning
In this analysis, time binning is used to capture the cyclical and seasonal patterns inherent in criminal behavior. Crime often follows temporal trends—certain types of incidents may be more likely to occur at specific times of day, days of the week, or during particular seasons. For example, incidents related to nightlife might peak during late evening hours, while other crimes may correlate with daily commuter traffic or specific days like weekends. By extracting temporal features like `YEAR`, `DAY_OF_YEAR`, `DAY_OF_WEEK`, and a continuous `TIME` (floating-point hour), we enable the model to identify these patterns and trends. This granular binning enhances the predictive power of the model, allowing it to detect not only broad time-based patterns but also more subtle nuances in how crime evolves throughout the day, week, and year.

#### Temporal Features Breakdown
* `YEAR`: Provides a clear yearly context, which can be crucial for identifying long-term trends or annual shifts in patterns.
* `TIME` (floating point): Offers a precise measure of the time of day, allowing models to pick up on subtle temporal shifts within a 24-hour cycle.
* `DAY_OF_YEAR`: Captures the day within the year, useful for identifying seasonal patterns or trends that recur annually.
* `DAY_OF_WEEK`: Encodes weekly cycles, helping the model detect patterns associated with specific weekdays.

In [15]:
# Convert FIRST_OCCURRENCE_DATE to Pandas date time format
df['FIRST_OCCURRENCE_DATE'] = pd.to_datetime(df['FIRST_OCCURRENCE_DATE'], format='%m/%d/%Y %I:%M:%S %p')

# Extract date and time components
df['YEAR'] = df['FIRST_OCCURRENCE_DATE'].dt.year
# df['MONTH'] = df['FIRST_OCCURRENCE_DATE'].dt.month -- Removed in favor of DAY_OF_YEAR
# df['DAY_OF_MONTH'] = df['FIRST_OCCURRENCE_DATE'].dt.day -- Removed in favor of DAY_OF_YEAR
# df['HOUR'] = df['FIRST_OCCURRENCE_DATE'].dt.hour -- Removed in favor of a floating point hour describing the time of day with more precision
df['TIME'] = (
    df['FIRST_OCCURRENCE_DATE'].dt.hour + 
    df['FIRST_OCCURRENCE_DATE'].dt.minute / 60 + 
    df['FIRST_OCCURRENCE_DATE'].dt.second / 3600
)
df['DAY_OF_YEAR'] = df['FIRST_OCCURRENCE_DATE'].dt.dayofyear
df['DAY_OF_WEEK'] = df['FIRST_OCCURRENCE_DATE'].dt.dayofweek  # 0 = Monday, 6 = Sunday

# Finally, dropping the FIRST_OCCURRENCE_DATE column
df.drop(columns=['FIRST_OCCURRENCE_DATE'], inplace=True)

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=5))

,OFFENSE_TYPE_ID,GEO_LON,GEO_LAT,YEAR,TIME,DAY_OF_YEAR,DAY_OF_WEEK
363807,theft-items-from-vehicle,-104.962189,39.783827,2023,15.500000,27,4
22118,theft-items-from-vehicle,-104.927533,39.683551,2021,22.083333,347,0
173145,theft-parts-from-vehicle,-104.942203,39.716703,2022,18.000000,190,5
64890,criminal-mischief-mtr-veh,-105.038008,39.755626,2022,0.616667,47,2
300388,theft-other,-104.981333,39.709206,2023,10.000000,257,3


### Location Slicing & Binning 
In this step, we convert latitude and longitude into a 15-meter by 15-meter grid system using a projected coordinate system (UTM). This method ensures that each crime location is assigned to a unique, precise grid cell, facilitating accurate spatial analysis.

See <https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system>

#### Why This is Better Than Using Raw Latitude and Longitude

Using UTM coordinates overcomes the limitations of raw latitude and longitude, which vary in distance depending on location. UTM coordinates provide uniform distance measurements in meters, improving precision and making it easier to define consistent grid cells. This approach also simplifies spatial analysis by grouping data into fixed-size cells, making patterns like crime hotspots easier to identify and computationally more efficient to process.

In [16]:
# Initialize UTM projection (assuming UTM Zone 13N for Denver)
#   https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system
#   https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system#/media/File:Universal_Transverse_Mercator_zones.svg
utm_proj = pyproj.Proj(proj="utm", zone=13, datum="WGS84")

def get_grid_block(lat, lon, grid_size=15):
    try:
        # Convert latitude and longitude to UTM coordinates (X, Y)
        x, y = utm_proj(lon, lat)  # Correct order: lon, lat
        
        # Calculate the X and Y blocks
        x_block = int(x // grid_size)
        y_block = int(y // grid_size)
        
        return x_block, y_block
    except:
        # If there is an issue parsing, return -1
        return -1, -1

# Apply the function to the DataFrame
df[['X_BLOCK', 'Y_BLOCK']] = df.apply(lambda row: get_grid_block(row['GEO_LAT'], row['GEO_LON']), axis=1, result_type='expand')

# Drop rows where X_BLOCK or Y_BLOCK is -1
df = df[(df['X_BLOCK'] != -1) & (df['Y_BLOCK'] != -1)]

# Note change in size
post_null_geo_block_removal_dataframe_shape = df.shape
print(f'Dataframe size before removal: {original_dataframe_shape[0]} items')
print(f'Dataframe size after removal: {post_null_geo_block_removal_dataframe_shape[0]} items')
print(f'--> {original_dataframe_shape[0] - post_null_geo_block_removal_dataframe_shape[0]} items removed!')

# Finally, dropping the GEO_LAT and GEO_LON columns since we have replaced them with X_BLOCK and Y_BLOCK
df.drop(columns=['GEO_LAT'], inplace=True)
df.drop(columns=['GEO_LON'], inplace=True)

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=5))

Dataframe size before removal: 394736 items
Dataframe size after removal: 394475 items
--> 261 items removed!


,OFFENSE_TYPE_ID,YEAR,TIME,DAY_OF_YEAR,DAY_OF_WEEK,X_BLOCK,Y_BLOCK
159533,robbery-business,2022,11.750000,210,4,33909,293263
326482,theft-from-mails,2020,20.350000,151,5,33120,293532
316099,theft-other,2020,12.333333,357,1,33398,292925
209408,criminal-mischief-other,2024,17.000000,209,5,33266,292860
339235,fraud-by-telephone,2020,13.000000,168,1,33164,292904


In this section, we are processing the geographical data to assign each crime incident to a specific grid cell based on its latitude and longitude. We achieve this by converting the coordinates to UTM projection, which provides more accurate distance-based calculations than traditional latitude and longitude. For each point, we calculate the corresponding grid cell by dividing the UTM coordinates by the grid size (15 meters) and then storing the results as `X_BLOCK` and `Y_BLOCK`.

However, if any coordinates cannot be processed correctly (due to projection issues or invalid data), we assign -1 as a placeholder. After applying this transformation, we remove any rows where either the `X_BLOCK` or `Y_BLOCK` is -1, indicating invalid grid assignments. The sizes of the DataFrame before and after this removal are printed to highlight the impact of cleaning the data. This step ensures that only valid data points are included for further analysis and modeling.

### Adding in OpenStreetMap Features
In this section, we aim to enrich our dataset with spatial features from OpenStreetMap (OSM). By querying OSM data for geographic features based on the centroids of the X and Y grid blocks (calculated from latitude and longitude), we can incorporate valuable context such as the proximity to roads, points of interest, or land use types. This spatial data enhances our analysis by providing a richer understanding of the environment in which the crimes occur, potentially improving predictive modeling. By leveraging OpenStreetMap data, we can gather additional information about the geographical characteristics of each grid cell, allowing for more nuanced predictions based on spatial context.

In [23]:
# Import tqdm for progress bar
from tqdm import tqdm

# Re-define the UTM projection from earlier
utm_proj = pyproj.Proj(proj="utm", zone=13, datum="WGS84")
latlon_proj = pyproj.Proj(proj="latlong", datum="WGS84")

# Create a transformer for converting UTM to lat/lon
transformer = pyproj.Transformer.from_proj(utm_proj, latlon_proj)

# Southwest and Northeast corners in latitude and longitude
# southwest_lat, southwest_lon = 36.9934, -109.0452 # (near the Four Corners region) -- Removed, trying to do the entire state uses an insane amount of RAM
# northeast_lat, northeast_lon = 41.0034, -102.0415 # (near the Kansas border) -- Removed, trying to do the entire state uses an insane amount of RAM
southwest_lat, southwest_lon = 39, -105
northeast_lat, northeast_lon = 40, -104

# Define block size in meters (15 meters as mentioned)
block_size = 15

# Calculate number of blocks along X and Y
num_x_blocks = int(area_width // block_size)
num_y_blocks = int(area_height // block_size)

# Calculate total number of blocks
total_blocks = num_x_blocks * num_y_blocks
print(total_blocks)

# Create a grid representing those blocks in numpy
grid = np.empty((num_y_blocks, num_x_blocks), dtype=object)

print(f'Created map block/cell grid: {num_y_blocks} rows x {num_x_blocks} columns = {total_blocks} blocks')

# Function to calculate the centroid of a block (in UTM)
def get_centroid(x_block, y_block, grid_size=15):
    # Get the bottom-left corner coordinates in UTM
    x_origin, y_origin = x_block * grid_size, y_block * grid_size
    
    # Calculate the centroid by moving half the grid size in both directions
    x_centroid = x_origin + grid_size / 2
    y_centroid = y_origin + grid_size / 2
    
    # Convert UTM coordinates to latitude and longitude
    lon, lat = transformer.transform(x_centroid, y_centroid)
    
    return lat, lon

# Initialize the progress bar
progress_bar = tqdm(total=total_blocks, desc="Identifying Block Centroids", position=0, ncols=120)

# Loop through each block in the grid and calculate the centroid
for y_block in range(num_y_blocks):
    for x_block in range(num_x_blocks):
        # Initialize the block with an empty dictionary
        grid[y_block, x_block] = {}

        # Calculate the centroid for the current block
        lat, lon = get_centroid(x_block, y_block, block_size)
        
        # Store the centroid in the grid at the corresponding block position
        grid[y_block, x_block]['centroid'] = (lat, lon)

        # Manually update the progress bar after each block
        progress_bar.update(1)

# Close the progress bar once finished
progress_bar.close()

-7386250


ValueError: negative dimensions are not allowed